# Predicting Energy Price using Netezza Performance Server in-built functions

In this notebook we will be examining energy data and predict the energy price based on parameters like temperature, pressure, humidity,wind_speed,precipitation. Netezza INZA functions and
in-databse analytics allows us to preform anaysis on very large datasets without bringing the data aback to the client. We will be using Netezza in-built functions to achieve this.


# Install pre-requisites

In [ ]:
pip install nzpy

## Connection to Netezza

Before we apply functions to the data, we will connection to NPS and analyze the data. Replace either the `<NAME OF CONNECTION>` OR the connection parameters below:

In [ ]:
# Setup connection and use the credentials from the connection. Replace the following values before you start

from project_lib import Project
project = Project.access()
NPS_credentials = project.get_connection(name="NPS")

username=NPS_credentials['username']
password=NPS_credentials['password']
host=NPS_credentials['host']
database=NPS_credentials['database']

## OR


# username="<username>"
# password="<password>"
# host="<hostname or ip>"
# database="system"

In [ ]:
import nzpy
import os
import pandas as pd

db = 'NZPY_TEST'
con = nzpy.connect(user=username, password=password, host=host,
                   database=database, port=5480)
selectQuery = f"select 1 from _v_database where database = '{db}'"
createQuery = f"create database {db}"
## Make sure the database exists, if not create one
with con.cursor() as cur:
    cur.execute(selectQuery)
    r = cur.fetchone();
    if r is None:
        cur.execute(createQuery)
        
# Now connect using the new database.        
con = nzpy.connect(user=username, password=password, host=host,
                   database=db, port=5480)
cursor=con.cursor()



## Load Energy Price data to Netezza
We will load the csv file data to Netezza table using `external table` feature of Netezza.

In [ ]:
## initialize cursor
cursor=con.cursor()
## drop table if exists
table='energy_price'
cursor.execute(f'drop table {table} if exists')

cursor.execute('''
CREATE TABLE nzpy_test..energy_price (
    temperature    REAL,
    pressure    REAL,
    humidity    REAL,
    wind_speed    REAL,
    precipitation    REAL,
    price    REAL,
    price_hour    TIMESTAMP
)
''')
print('Table energy price successfully created')
## Load the data to Netezza

with con.cursor() as cursor:
    cursor.execute('''
        insert into nzpy_test..energy_price
            select * from external '/project_data/data_asset/energy_price.csv'
                using (
                    delim ',' 
                    remotesource 'odbc'
                    )''')
    print(f"{cursor.rowcount} rows inserted")

In [ ]:
query = 'select * from nzpy_test..ENERGY_PRICE'
energyData = pd.read_sql(query, con)
energyData.head()

## Data Visualization

Data visualization can be used to find patterns, detect outliers, understand distribution and more. We can use graphs such as:

    Histograms, boxplots, etc: To find distribution / spread of our continuous variables.
    Bar charts: To show frequency in categorical values.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
sns.set(style="darkgrid")
sns.set_palette("hls", 3)

In [ ]:
TARGET_LABEL_COLUMN_NAME = 'PRICE_HOUR'
columns_idx = np.s_[0:] # Slice of first row(header) with all columns.
first_record_idx = np.s_[0] # Index of first record

string_fields = [type(fld) is str for fld in energyData.iloc[first_record_idx, columns_idx]] # All string fields
all_features = [x for x in energyData.columns if x != TARGET_LABEL_COLUMN_NAME]
categorical_columns = list(np.array(energyData.columns)[columns_idx][string_fields])
categorical_features = [x for x in categorical_columns if x != TARGET_LABEL_COLUMN_NAME]

print('All Features: ', all_features)
print('\nCategorical Features: ', categorical_features)
print('\nAll Categorical Columns: ', categorical_columns)

In [ ]:
plotDf = energyData
plotDf.columns = [c.decode().lower() for c in plotDf.columns]

In [ ]:
## Check the data types of the columns
print(plotDf.dtypes)

In [ ]:
plotDf['dates'] = pd.to_datetime(plotDf['price_hour'], format='%Y-%m-%d').dt.floor('D')
updDf = plotDf.drop(columns=['price_hour'])

updDf.head()

In [ ]:
updDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

Now lets add visualization showing correlation between Price and other factors that affects the price (temperature, pressure, humidity, wind speed and precipitation)

In [ ]:
tempPriceDf = updDf[["temperature","price","dates"]]
tempPriceDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

tempPriceDf = updDf[["pressure","price","dates"]]
tempPriceDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

tempPriceDf = updDf[["humidity","price","dates"]]
tempPriceDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

tempPriceDf = updDf[["wind_speed","price","dates"]]
tempPriceDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

tempPriceDf = updDf[["precipitation","price","dates"]]
tempPriceDf.groupby('dates').sum().plot.line().legend(loc='upper left',bbox_to_anchor=(1.05, 1))

## Analyzing the data

First of all we want to understand our data. For this we will run some descriptive statisctics
the SUMMARY1000 call analyzes all fieldsin a table by default and places the output in a new table, which we then query

The `SUMMARY1000` call analyzes all fields in the table by default and places the output in a new table, which we can query to get results.

In [ ]:
# cursor.execute("drop table PRICE_TEMP_ANALYSIS if exists") #drop the table if it exists - keep our database clean
# cursor.execute("drop table PRICE_TEMP_ANALYSIS_NUM if exists")
# cursor.execute("drop table PRICE_TEMP_ANALYSIS_TIMESTAMP if exists")

# cursor.execute("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS')");
# cursor.execute("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS_NUM')")
# cursor.execute("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS_TIMESTAMP')")

pd.read_sql("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS')", con)
pd.read_sql("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS_NUM')",con)
pd.read_sql("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS_TIMESTAMP')",con)


summaryDF = pd.read_sql("CALL nza..SUMMARY1000('intable=ENERGY_PRICE, outtable=PRICE_TEMP_ANALYSIS');", con)
summaryAnalysisDF = pd.read_sql('select * from PRICE_TEMP_ANALYSIS', con)
summaryAnalysisDF.head()



Let's find if there is a relationship between `temperature` and `price` in the data. we are looking for covariance and NPS has inbuilt function calle4d COV to get the results.

In [ ]:
# cursor.execute("drop table PRICE_TEMP_ANALYSIS if exists")
pd.read_sql("CALL nza..DROP_TABLE('PRICE_TEMP_ANALYSIS')",con);

# use the Covariance function, store results in PRICE_TEMP_ANALYSIS
pd.read_sql("CALL nza..COV('intable=ENERGY_PRICE, incolumn=TEMPERATURE;PRICE,outtable=PRICE_TEMP_ANALYSIS');",con)
# bring the results table into the notebook - or just query it directly in Netezza
pd.read_sql('select * from PRICE_TEMP_ANALYSIS', con)

## Training Data

In [ ]:
# clean up the analysis tables
# cursor.execute("drop table PRICE_TEMP_NEW if exists")
pd.read_sql("CALL nza..DROP_TABLE('PRICE_TEMP_NEW')",con);
# the INZA functions usully need a unique ID for each row of data, we use the internal ROWID for this
cursor=con.cursor()
cursor.execute("create table PRICE_TEMP_NEW as select *,DATE(PRICE_HOUR) AS DAY,ROWID as ID from ENERGY_PRICE")
priceTempNewDf = pd.read_sql('select * from PRICE_TEMP_NEW limit 10', con)
# TO_DATE( PRICE_HOUR,'YYYY-MM-DD HH24:MI:SS'); 

In [ ]:
priceTempNewDf.head()

## Create Model using Timeseries algorithm

In [ ]:
#drop model if it was already created. Initially you might want to comment this out
# and run as it throws error if if doesn't find the model
# cursor.execute("CALL nza..DROP_MODEL('model=PRICE_TIME');")

# we now call a timeseries algorithm to create a model, the model name is PRICE_TIME
pd.read_sql("CALL nza..TIMESERIES('model=PRICE_TIME, intable=ADMIN.PRICE_TEMP_NEW, by=DAY, time=PRICE_HOUR, target=PRICE' );",con)

In [ ]:
# we can list our models here
pd.read_sql("select * from v_nza_models;",con=con)

The `NZA_META_<model_name>_SERIES` contains information pertaining to the entire time series.
The table contains one line for each time series in the input data with the following columns.

In [ ]:
pd.read_sql("select * from NZA_META_PRICE_TIME_SERIES;", con=con)

The `NZA_META_<model_name>_FORECAST` table holds forecast values. The table contains one line
for each time series and point in time for which a forecast has been made, with the following
columns.

In [ ]:
pd.read_sql("select * from NZA_META_PRICE_TIME_FORECAST;", con=con)

In [ ]:
## model appears 
# the process store a huge amount of data about the models in metadata tables
pd.read_sql("CALL nza..PRINT_TIMESERIES('model=PRICE_TIME, history=true');", con)

In [ ]:
# and some simple charts
cursor.execute("CALL nza..PRINT_TIMESERIES('model=PRICE_TIME, history=true, series=sinus');")

In [ ]:
con.close()